---

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [ ]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/e8/d9/104988573fd2c1acdc64e66883b35fb8ae559310d2d9f77db78bf7de9add/gensim-4.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 21.1 MB/s eta 0:00:00m eta 0:00:010:01:01


In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)
wv_embeddings_google_news = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

В качестве второго предобученного датасета с эмбеддингами берется датасет с kaggle, в котором хранятся гугл новости

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)
    word = 'dog'
if word in wv_embeddings_google_news:
    print(wv_embeddings_google_news[word].dtype, wv_embeddings_google_news[word].shape)

float32 (200,)
float32 (300,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")
print(f"Num of words in google news: {len(wv_embeddings_google_news.index_to_key)}")

Num of words: 1787145
Num of words in google news: 3000000


Датасет гугл нвостей больше почти в 2 раза

Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [ ]:
top_5_similar_words = wv_embeddings.most_similar('dog')[:5]
top_5_similar_words_google = wv_embeddings_google_news.most_similar('dog')[:5]

cat_in_top_5 = any(word == 'cat' for word, _ in top_5_similar_words)
cat_in_top_5_google = any(word == 'cat' for word, _ in top_5_similar_words_google)

if cat_in_top_5:
    position = [i+1 for i, (word, _) in enumerate(top_5_similar_words) if word == 'cat'][0]
    print(f"Слово 'cat' входит в топ-5 близких слов к 'dog' на позиции {position}.")
else:
    print("Слово 'cat' не входит в топ-5 близких слов к 'dog'.")

if cat_in_top_5_google:
    position = [i+1 for i, (word, _) in enumerate(top_5_similar_words_google) if word == 'cat'][0]
    print(f"Слово 'cat' входит в топ-5 google news близких слов к 'dog' на позиции {position}.")
else:
    print("Слово 'cat' не входит в топ-5 google news близких слов к 'dog'.")
print(top_5_similar_words)
print(top_5_similar_words_google)


Слово 'cat' не входит в топ-5 близких слов к 'dog'.
Слово 'cat' входит в топ-5 google news близких слов к 'dog' на позиции 5.
[('animal', 0.8564180135726929), ('dogs', 0.7880866527557373), ('mammal', 0.7623804211616516), ('cats', 0.7621253728866577), ('animals', 0.760793924331665)]
[('dogs', 0.8680489659309387), ('puppy', 0.8106428384780884), ('pit_bull', 0.780396044254303), ('pooch', 0.7627376914024353), ('cat', 0.7609457969665527)]


В датасете со StackOverflow слово cat не входит топ 5 слов близких слов к слову dog. Это можно объяснить тем, что вопросы на StackOverflow ориентированы на программные решения.

Что касается датасета с новостями, то слово cat входит в топ 5 близких слов на 5 позиции.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

In [ ]:
import numpy as np

def question_to_vec(question, embeddings, tokenizer, dim=200):

    tokens = tokenizer.tokenize(question)
    vec = np.zeros(dim)
    word_count = 0

    for token in tokens:
        if token in embeddings:
            vec += embeddings[token]
            word_count += 1

    if word_count > 0:
        vec /= word_count

    return vec

question = "What's up, doc?"
vec = question_to_vec(question, wv_embeddings, tokenizer, dim=200)
print(vec)



[ 0.04758342  0.1043864   1.61915767  0.8074016  -1.99182165  0.06548174
 -1.46828127  0.21234712  0.86960089 -2.25627613 -1.53800523 -1.72828186
 -0.98838788 -0.0647449  -2.07048965 -0.34603521 -2.31884408  1.32516062
  0.57407027  2.48408914 -0.3021597   1.80067503  1.80627775 -0.86563927
 -0.65428364 -0.5026263   0.44397047  2.26097298 -1.36881125 -2.09525418
  0.45501301 -1.67106044  0.67697126 -0.26760721 -0.70426261 -1.18388033
  0.84861434 -1.87027824  0.72793448  2.27651691 -1.16022742 -1.46120775
  0.90104181 -1.37618864 -0.19955006  0.17984794  0.28916872  0.28367925
  1.35920763  3.56088662 -1.2192415  -0.39393994 -0.6585961  -3.7409966
  0.62410444 -1.86811972  1.67136621  0.52260935 -3.47729373  0.54318392
  0.772609    0.13898942 -0.56915224  0.17635784  0.88446689  0.66217452
 -0.11441653 -0.233316    0.0341717  -2.81008863  1.3904984   2.06808424
  1.55512798  1.78024185  1.34696269  1.78097987  1.57226515  1.5728122
  0.10570773 -1.20426238  3.27850366  0.52439314  0.0

In [ ]:
question = "What's up, doc?"
vec = question_to_vec(question, wv_embeddings_google_news, tokenizer, dim=300)
print(vec)

[-0.06887817  0.13851929  0.09716797  0.11700439 -0.07254028  0.02880859
  0.08892822 -0.08273315  0.08148193 -0.03393555 -0.08176422 -0.15527344
 -0.03479004  0.01931763 -0.12231445  0.0869751   0.06078339  0.09515381
 -0.02400208 -0.08093262  0.00848389  0.09790039  0.05557251 -0.00257874
  0.03014946 -0.00592041 -0.09899902 -0.02862549  0.01922607  0.02783203
 -0.03529167  0.06925964 -0.12582397 -0.07250977 -0.09442139  0.05395508
 -0.14233398  0.08947754  0.10870361  0.11499023  0.01660156 -0.13366699
  0.06329346  0.04675293 -0.04379272  0.02703857 -0.00421143 -0.02893066
  0.13220215  0.08950806 -0.0592041   0.00683594 -0.08010864  0.01846313
  0.05725098 -0.03033447 -0.04376221 -0.00234985  0.04919434 -0.0602417
 -0.0344429   0.0259552  -0.07002127  0.00262451 -0.07403564 -0.10693359
 -0.04577637  0.03259277  0.04638672 -0.03561401  0.05285358  0.06915283
  0.06201172  0.07174683 -0.12792969  0.04211426  0.07662964  0.03735352
  0.01635742  0.16152954  0.02600098 -0.05038452 -0.

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
input_sentence = "I love neural networks"

vector_from_sentence_stack = question_to_vec(input_sentence, wv_embeddings, tokenizer)
third_component_stack = round(vector_from_sentence_stack[2],2)

vector_from_sentence_google = question_to_vec(input_sentence, wv_embeddings_google_news, tokenizer, dim=300)
third_component_google = round(vector_from_sentence_google[2],2)


print(f'Третья компонента вектора Stackoverflow {third_component_stack}')
print(f'Третья компонента вектора Google News {third_component_google}')
third_component_google

Третья компонента вектора Stackoverflow -1.29
Третья компонента вектора Google News 0.02


0.02

In [ ]:
sentence = "I love neural networks"

tokens = tokenizer.tokenize(sentence)

vectors_stack = [wv_embeddings[word] for word in tokens if word in wv_embeddings]
vectors_google = [wv_embeddings_google_news[word] for word in tokens if word in wv_embeddings_google_news]

combined_vector_stack = np.mean(vectors_stack, axis=0)
combined_vector_google = np.mean(vectors_google, axis=0)

third_component_stack = combined_vector_stack[2]
third_component_google = combined_vector_google[2]

third_component_rounded_stack = round(third_component_stack, 2)
third_component_rounded_google = round(third_component_google, 2)

print(f'Третья компонента вектора Stackoverflow {third_component_rounded_stack}')
print(f'Третья компонента вектора Google News {third_component_rounded_google}')

Третья компонента вектора Stackoverflow -1.2899999618530273
Третья компонента вектора Google News 0.019999999552965164


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Если мы хотим найти максимальную разницу между `Hits@47 и DCG@1`, то тогда Hits@47 должно быть максимальным, а DCG@1 должно быть минимальным.

Максимальное значение для Hits@47 это 1 при условии, что для каждого запроса ранг дубликата <= 47.

Минимальное значение для DCG@1 это 0 при условии, что для каждого запроса ранг дубликата > 1.

`Ответ: максимальная разница будет равна 1.`

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [ ]:
import math
rank = 9
K= 10
DCG = 1/math.log2(1+rank)*(rank <= K)
print(f"DCG@10, если rank_qi' = 9 равно {round(DCG,1)}")

DCG@10, если rank_qi' = 9 равно 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    hits_value = 0
    for dup_rank in dup_ranks:
        if dup_rank <= k:
            hits_value += 1
    hits_value /= len(dup_ranks)
    return hits_value

In [ ]:
def dcg_score(dup_ranks, k):
    dcg_value = 0
    for dup_rank in dup_ranks:
        if dup_rank <= k:
            dcg_value += 1 / math.log(1 + dup_rank, 2)
    dcg_value /= len(dup_ranks)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown"]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

def find_dup_ranks(candidates, copy_answer):
    for i, candidate in enumerate(candidates, 1):
        if candidate == copy_answer:
            return i

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [find_dup_ranks(sentence, copy_answers[i]) for i, sentence in enumerate(candidates_ranking)]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
import numpy as np
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split("\t"))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vector = question_to_vec(question, embeddings, tokenizer, dim).reshape(1,-1)
    candidate_vectors = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates]

    cosine_range = [cosine_similarity(question_vector, candidate_vec.reshape(1,-1)) for candidate_vec in candidate_vectors]
    ranked_candidates = [(i, candidate, cosine_range[i]) for i, candidate in enumerate(candidates)]
    ranked_candidates.sort(key=lambda x: x[2], reverse=True)

    return [(position, candidate) for position, candidate, _ in ranked_candidates]


Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.386
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@1000: 0.434 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
words = [question for question in train_data]

In [ ]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=5).wv

In [ ]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 1.000 | Hits@   1: 1.000
DCG@   5: 1.000 | Hits@   5: 1.000
DCG@  10: 1.000 | Hits@  10: 1.000
DCG@ 100: 1.000 | Hits@ 100: 1.000
DCG@ 500: 1.000 | Hits@ 500: 1.000
DCG@1000: 1.000 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
